## Importando bibs


In [2]:
import argparse
import time
from pathlib import Path
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

c:\Users\marcos\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
#Import das funções auxiliares.
import auxiliares
import os

In [3]:
# Transcrevendo valores defaults e valores inseridos no terminal
project = "runs/detect"
trace = True
save_txt= True
save_img = True
device = ""
weights = "yolov7-e6e.pt"

In [4]:
 # Initialize
set_logging()
device = select_device(device)
half = device.type != 'cpu'  # half precision only supported on CUDA


YOLOR  v0.1-114-g8b616af torch 1.12.1+cpu CPU



In [5]:
 # Load model
imgsz = 640
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size

Fusing layers... 


Model Summary: 792 layers, 151687420 parameters, 151687420 gradients


In [6]:
img_size = 640

In [7]:
if trace:
    model = TracedModel(model, device, img_size)

if half:
    model.half()  # to FP16

 Convert model to Traced-model... 


c:\Users\marcos\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:482.)
  return self._grad


 traced_script_module saved! 
 model is traced! 



In [8]:
# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

In [9]:
# Run inference
def run_inference(source, imgsz, device, model, name = "exp", augment = True, agnostic = True, conf_thres = 0.25,
                  iou_thres = 0.45, classes=None, view_img = False, save_img = True, save_txt = True):
    
    # Directories
    save_dir = Path(increment_path(Path(project) / name, exist_ok=False))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
    
     # Set Dataloader
    vid_path, vid_writer = None, None
    dataset = LoadImages(source, img_size=imgsz, stride=stride)
    
    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment)[0]

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic)
        t3 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            #if webcam:  # batch_size >= 1
               #p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            #else:
                #p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
            p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if True else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:

                            # new = np.asarray(xyxy)
                            # f.write(f"{str(new[0])} {str(new[1])} {str(new[2])} {str(new[3])}") #modified.
                            # f.write("----")
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)
                        
            # Print time (inference + NMS)
            print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                    print(f" The image with the result is saved in: {save_path}")
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        #print(f"Results saved to {save_dir}{s}")

In [11]:
#Listando pastas que serão acessadas.
dogsBreeds = auxiliares.retornaListaDeConteudoNoDiretorio("Dogs")

In [12]:
#Rodando yolov7 em todas as 590 pastas com as raças dos cachorros.
with torch.no_grad():
    for breed in dogsBreeds:
        try:
            run_inference("./Dogs/" + breed, 640, device, model)
        except:
            print("Error in the folder")

c:\Users\marcos\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1 dog, Done. (2299.0ms) Inference, (209.0ms) NMS
 The image with the result is saved in: runs\detect\exp18\0.220px-Affenpinscher.jpg
1 cat, Done. (2084.3ms) Inference, (1.9ms) NMS
 The image with the result is saved in: runs\detect\exp18\1.220px-Affenpinscher_circa_1915.jpg
1 dog, Done. (1136.9ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp18\2.170px-Borismindre.jpg
1 dog, Done. (1868.6ms) Inference, (2.0ms) NMS
 The image with the result is saved in: runs\detect\exp19\0.220px-Afghan_dog_-_cropped.jpg
1 horse, Done. (1703.7ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp19\1.220px-Stamp_of_Afghanistan_-_1962_-_Colnect_647962_-_Afghan_Hound_Canis_lupus_familiaris.jpeg
1 dog, Done. (1395.3ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs\detect\exp19\2.220px-Afghan_Hound3.jpg
2 dogs, Done. (1604.6ms) Inference, (21.0ms) NMS
 The image with the result is saved in: runs\detect\exp19\3.220px-Small_Dog

In [14]:
#Definindo caminho até pasta com os "exps".
caminho = "C:\\Users\\marcos\\yolov7\\runs\\detect"

#Salvamos em uma lista todos o nome de todas as pastas "exps" que temos no "caminho".
exps = auxiliares.retornaListaDeConteudoNoDiretorio(caminho)

for exp in exps:

    #Contador para alterar entre imagens.
    contadorImagens = 0

    #Cria pasta para salvar imagens.
    os.mkdir(caminho + "\\" + exp + "\\imagens-cortadas")

    #Salva caminho para ser utilizado.
    dirImagensCortadas = caminho + "\\" + exp + "\\imagens-cortadas"

    #Primeiro lemos o as imagens contidas na pasta.
    imagens = auxiliares.retornaListaDeConteudoNoDiretorio(caminho + "\\" + exp)
    
    #Removemos o elemento "labels" da lista.
    imagens.remove("labels") #Se for certeza que "labels" é último elemento, posso trocar para (len - 1).

    #Agora pegamso as os txts dentro de "labels".
    txts = auxiliares.retornaListaDeConteudoNoDiretorio(caminho + "\\" + exp + "\\labels")

    for txt in txts:
        
        #Lendo conteúdo do arquivo de texto.
        conteudoDoTxt = auxiliares.retornaListaComConteudoDoArquivoDeTexto(caminho + "\\" + exp + "\\labels\\" + txt)

        #Percorrendo linhas do arquivos texto.
        for linha in conteudoDoTxt:
            
            #Só corta se for da classe cachorro.
            if linha[0] == "16":
                
                auxiliares.cortaRetanguloIdentificadoPelaYolo(linha, caminho + "\\" + exp + "\\" + imagens[contadorImagens], imagens[contadorImagens], dirImagensCortadas)
        
        #Incrementa contador para pegar próxima imagem.
        contadorImagens += 1